# large-scale-route-optimization

In [10]:
import pandas as pd
import numpy as np
import random, math
from datetime import datetime

# -------------- 数据加载 --------------
df_dist = pd.read_csv('distance.csv')
# city ↔ idx
cities = pd.unique(df_dist[['Source','Destination']].values.ravel())
c2i = {c:i for i,c in enumerate(cities)}
n_city = len(cities)
# 建矩阵
D = np.full((n_city,n_city), 1e9)
for _,r in df_dist.iterrows():
    i,j = c2i[r.Source], c2i[r.Destination]
    D[i,j] = D[j,i] = r['Distance(M)']  

df_ord = pd.read_csv('order_small.csv',
                     parse_dates=['Available_Time','Deadline'])
# depot 固定第 0 个
depot = c2i[df_ord.Source.iloc[0]]
orders = []
t0 = df_ord.Available_Time.min()
for _,r in df_ord.iterrows():
    dst = c2i[r.Destination]
    w = r.Weight
    ready = (r.Available_Time - t0).total_seconds()
    dl = (r.Deadline - t0).total_seconds()
    orders.append({'dst':dst, 'w':w, 'ready':ready, 'dl':dl})

# -------------- 参数 --------------
M = len(orders)
K = 8                 # 车辆数
CAP = 1e8             # 单车载重
PENALTY_OVER = 1e6    # 超载惩罚
PENALTY_LATE = 1e3    # 超时惩罚

# -------------- 初始解：随机拆分订单到 K 辆车 --------------
def init_solution():
    sol = [[] for _ in range(K)]
    for i in range(M):
        sol[random.randrange(K)].append(i)
    return sol

# -------------- 评价函数 --------------
def eval_solution(sol):
    total = 0
    for route in sol:
        load = 0
        t = 0
        cur = depot
        for oid in route:
            o = orders[oid]
            total += D[cur, o['dst']]
            t += D[cur, o['dst']]
            load += o['w']
            # 时间窗惩罚
            if t < o['ready']:
                t = o['ready']
            if t > o['dl']:
                total += PENALTY_LATE * (t - o['dl'])
            cur = o['dst']
        total += D[cur, depot]  # 回仓库
        # 载重惩罚
        if load > CAP:
            total += PENALTY_OVER * (load - CAP)
    return total

# -------------- 邻域：在两条路径中随机交换一个订单 --------------
def neighbor(sol):
    new = [r.copy() for r in sol]
    # 选两辆车
    a,b = random.sample(range(K),2)
    if not new[a] or not new[b]:
        return new
    i = random.randrange(len(new[a]))
    j = random.randrange(len(new[b]))
    new[a][i], new[b][j] = new[b][j], new[a][i]
    return new

# -------------- 模拟退火 --------------
def simulated_annealing(T0=1e5, alpha=0.995, iters=50000):
    cur_sol = init_solution()
    cur_val = eval_solution(cur_sol)
    best_sol, best_val = cur_sol, cur_val
    T = T0
    for it in range(iters):
        nxt = neighbor(cur_sol)
        v = eval_solution(nxt)
        dE = v - cur_val
        if dE < 0 or random.random() < math.exp(-dE/T):
            cur_sol, cur_val = nxt, v
            if cur_val < best_val:
                best_sol, best_val = cur_sol, cur_val
        T *= alpha
        if T < 1e-3:
            break
    return best_sol, best_val

if __name__ == '__main__':
    sol, val = simulated_annealing()
    print("Best cost:", val)
    for k,route in enumerate(sol):
        print(f"Vehicle {k}:", route)


Best cost: 21106187275.0
Vehicle 0: [3, 9]
Vehicle 1: [6, 4]
Vehicle 2: []
Vehicle 3: [1, 7]
Vehicle 4: [2, 8]
Vehicle 5: [0, 5]
Vehicle 6: []
Vehicle 7: []
